# INFO 4271 - Group Project

Issued: June 11, 2024

Due: July 22, 2024

Please submit a link to your code base (ideally with a branch that does not change anymore after the submission deadline) and your 4-page report via email to carsten.eickhoff@uni-tuebingen.de by the due date. One submission per team.

---

# 1. Web Crawling & Indexing
Crawl the web to discover **English content related to Tübingen**. The crawled content should be stored locally. If interrupted, your crawler should be able to re-start and pick up the crawling process at any time.

In [7]:
#### Imports ####
import requests
from boilerpy3 import extractors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import re
from trafilatura import fetch_url, extract
import contractions
from nltk.stem import WordNetLemmatizer
from duplicateCheck import check_simhash,computeHash
import math
import numpy as np
from sklearn.cluster import KMeans
import json
import os
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.robotparser import RobotFileParser as rp
from bs4 import BeautifulSoup as bs
import html5lib
import requests
import stopit
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [8]:
#### preprocesses a term t 
#returns preprocessed t or False in case term is uninformative
def preprocess(t, tag):
 #get nltk stopwords english and german! (often also german stopwords still contained)
 stop_words = set(stopwords.words('english')).union(set(stopwords.words('german')))

 #initialize lemmatizer
 lemmatizer = WordNetLemmatizer()

 #convert to lowercase
 t = t.lower()

 #is url (starting with http(s) or www.)
 if re.compile(r'https?://\S+|www\.\S+').match(t):
    return False
      
 # special character,punctuation -> continue in this case 
 if re.compile(r'[^a-zA-Z0-9äöüÄÖÜ\s]').match(t):
    return False
 
 #is stopword
 if t in stop_words:
    return False 
 
 #remove also special chars inside token
 t = re.sub(r'[^a-zA-Z0-9äöüÄÖÜ\s]', '', t)
      
 #convert pos tag for lemmatization
 ltag = tag[0].lower()
 if ltag in ['a', 'r', 'n', 'v']:
          t = lemmatizer.lemmatize(t, ltag)

 return t

In [9]:
#### Indexing ####

#Add a document to the index. You need (at least) two parameters:
    #doc: The document to be indexed.
    #index: The location of the local index storing the discovered documents.
def index(doc, index):
 url = doc['url']
 indexPath = index

 #get index of index.json
 with open(indexPath, 'r', encoding='utf-8') as f:
     index = json.load(f)
     docID = len(index[1])
     print(docID)
     doc = doc['doc']
     f.close()


####  Text Preprocessing of the doc ###################

 #extract relevant text from raw html doc
 content = extract(doc)


###storing unprocessed text content ##### -> store as file at end if no duplicate
 docContents={}
 soup = bs(doc, 'html.parser')
 title = soup.find('title').text if soup.find('title') else 'No title'
 docContents['title'] = title
 meta_desc = soup.find('meta', attrs={'name': 'description'})

 ###if description available save description, else save first 40 words (or less if doc shorter) -> remove special characters from that ####
 if meta_desc:
     docContents['text'] = meta_desc['content'] 
 else:
    splittedText = content.split()
    if len(splittedText) >= 40:
     docContents['text'] = re.sub(r'[^a-zA-Z0-9äöüÄÖÜ\s.,?!-]', '', ' '.join(content.split()[:40]))
    else: 
     docContents['text'] = re.sub(r'[^a-zA-Z0-9äöüÄÖÜ\s.,?!-]', '', ' '.join(splittedText[:len(splittedText)]))

 #docContents['content'] = content
 # Find the first image
 first_image = soup.find('img')
 
 
 #convert to lowercase
 content = content.lower()

 #remove contractions:
 content = contractions.fix(content)
 

 #store words of the processed document
 processedDoc = []
 #tokenize, go through all words and do the steps for each at once
 for position, (t, tag) in enumerate(pos_tag(word_tokenize(content))):
    
      #break up hyphenated words:
       if '-' in t:
          t = t.split('-')
          firstTermPreprocessed = preprocess(t[0], tag)
          #if not uninformative
          if firstTermPreprocessed:
              processedDoc.append([firstTermPreprocessed, position])
          secondTermPreprocessed = preprocess(t[1], tag)
          if secondTermPreprocessed:
              processedDoc.append([secondTermPreprocessed, position])
       else:
           termPreprocessed = preprocess(t, tag)
           if termPreprocessed:
               processedDoc.append([termPreprocessed, position])

  ############# Near Duplicate checking ##############
  #near duplicate checking by means of the words in the processed document (processedDoc)

 docHash = computeHash([l[0] for l in processedDoc])
 # compare to other doc hashes
 if(check_simhash(docHash, index[2])):
     #break and dont index if is near duplicate
     return [False, 0]
 #if no duplicate save hash and insert terms in inverted index
 index[2].append(docHash)



############ Build up inverted index #####################
 for term, position in processedDoc:
     
      #entry in inverted index is: [docID, [occurence1, occurence2, ...]]
      #add on thy fly to inverted index
      #if word already in index add doc idto the words list
      if(term in index[0].keys()):
           #check if doc id already in list of that word (must be at the end)
           if index[0][term][len(index[0][term])-1][0] == docID:
                #if so append position there
                index[0][term][len(index[0][term])-1][1].append(position)
           #else add new list [docID, [position of t],skip pointer, tfidfval] for that word
           else:
               index[0][term].append([docID, [position], None, None])
               
      #if word not yet in index add "t: [[docID, [get position of t], tfidf weight for t in d, skip pointer (None) , tfidfval]]" to dict
      else:
           index[0][term] = [[docID, [position], None, None]]

 length = len(processedDoc)

 #add url , cluster of document (None so far) and length of preprocessed doc to list index[1] after indexing the doc
 index[1].append([url, None, length])

 #write changed index object to json
 with open(indexPath, 'w', encoding='utf-8') as f:
     json.dump(index, f, ensure_ascii=False)
 
 #save doc in documents ordner: name <docID>.json
 with open(os.path.join(os.getcwd(),"documents", str(docID) + ".json"), 'w', encoding='utf-8') as f:
     json.dump(docContents, f, ensure_ascii=False)
 
 #save first image of page if found
 if first_image:
     img_url = first_image['src']
     # Convert relative URL to absolute URL
     absolute_image_url = urljoin(url, img_url)
     f_ext = os.path.splitext(absolute_image_url)[-1]
     image_fetched = requests.get(absolute_image_url)
     if image_fetched.status_code == 200:
       # Get the content of the image
        image_content = image_fetched.content
       
        #save img in pictures ordner: name <docID>
        with open(os.path.join(os.getcwd(),"pictures", str(docID)) + f_ext, 'wb') as f:
           f.write(image_content)

 return [True, docID]
 
    
      

In [10]:
#### Clustering ####
#clusters the docs of the index and inserts the labels into the index (currently 30 clusters)
# cluster using kmeans clustering with tf-idf vectors
def cluster(index):
     print('start clustering')
     indexPath = index
     #get index of index.json
     f = open(indexPath, encoding='utf-8')
     index = json.load(f)
     #convert index to tf-idf vector representations for each document
     idx = index[0] 
     docs = index[1]
     #matrix to store vectors (rows: documents , cols: terms)
     tfIDFMatrix = np.zeros((len(docs), len(idx.keys())))
     print('buildMatrix')

     for t in range(len(idx.keys())):
         term = list(idx.keys())[t]
         for i in range(len(idx[list(idx.keys())[t]])):
             #term index: t, doc index: idx[t][i][0]

             docID = idx[term][i][0]
             
             #calculate tf (nr occurences of t in doc/ length of doc) * idf(log (#docs/ #docs containing t))
             # occurences of t in doc: len(idx[term][i][1])
             # length of doc: docs[docID][2]
             # # docs = len[docs]
             # #docs containing term = len(idx[term])
             tfValue = len(idx[term][i][1])/docs[docID][2]
             idfValue = math.log(len(docs)/len(idx[term]))
             tfIDFMatrix[docID][t] = tfValue * idfValue

             ## store idf value for that doc and term in index on the fly (for BM25 later on)
             index[0][term][i][3] = idfValue
     
     print('endBuildmatrix')

     #also calculate avg doc length and append to index
     docLengths = [l[2] for l in index[1]]
     avgLength = sum(docLengths)/len(docLengths)
     index[3] = avgLength
      
     #place skip pointers once after whole index is built up
     #### rearange skip pointers ###########
     #delete current skip pointers for that posting list
     for term in index[0].keys():
        # sqare |p| evenly spaced per posting list (p: length of posting list of term t)
        p = len(index[0][term])
        #just if posting list has at least length 4
        if(p >= 4):
            nrPointers = math.sqrt(p)
            spaceBetween = math.floor(p/nrPointers)
            #current index in postingslist
            i = 0
            while(i + spaceBetween < p):
                #set skip pointer [idx to jump to in postings list, docID at that index]
                index[0][term][i][2] = [i + spaceBetween, index[0][term][i + spaceBetween][0]]
                i += spaceBetween

     #store new  index with tf idf values 
     #write changed index object to json
     with open(indexPath, 'w', encoding='utf-8') as f:
      json.dump(index, f, ensure_ascii=False)
     # Initialize KMeans clustering
     print('clustering')
     num_clusters = 10  # Example: Number of clusters
     kmeans = KMeans(n_clusters=num_clusters, random_state=42)

     # Fit KMeans model to the TF-IDF matrix
     kmeans.fit(tfIDFMatrix)

     docLabels = kmeans.labels_
     #centroids of each cluster:
     centroids = kmeans.cluster_centers_
     #get term with highest tf-idf score per cluster centroid
     centroidTopics = []
     for centroid in centroids:
        centroidTopics.append(list(idx.keys())[np.argmax(centroid)])
     
     #insert labels in index
     for d in range(len(index[1])):
         index[1][d][1] = centroidTopics[docLabels[d]]
      
     print('endClustering')
        
      #write changed index object to json
     with open(indexPath, 'w', encoding='utf-8') as f:
      json.dump(index, f, ensure_ascii=False)
     
     return True

In [11]:
#### Crawling ####
#Crawl the web. You need (at least) two parameters:
#frontier: The frontier of known URLs to crawl. You will initially populate this with your seed set of URLs and later maintain all discovered (but not yet crawled) URLs here.
#index: The location of the local index storing the discovered documents. 
STORAGE_LOC = "index.json"
MAX_DOCS = 5
frontier = ["https://tuebingenresearchcampus.com/en/tuebingen",
            "https://tunewsinternational.com/category/news-in-english/",
            "https://www.tuebingen.de/en/",
            "https://uni-tuebingen.de/en/",
            "https://www.germany.travel/en/cities-culture/tuebingen.html",
            "https://www.iwm-tuebingen.de/www/en/index.html",
             "https://kunsthalle-tuebingen.de/en/",
             "https://www.opentable.com/food-near-me/stadt-tubingen-germany",
             "https://historicgermany.travel/historic-germany/tubingen/"
             "https://en.wikipedia.org/wiki/T%C3%BCbingen",
              "https://en.wikipedia.org/wiki/University_of_T%C3%BCbingen"]
links_seen = frontier.copy()

def crawl(frontier, indexPath):
    
    #get  first document of frontier while frontier not empty
    while len(frontier) != 0:
        with stopit.ThreadingTimeout(20) as context_manager:
            link = frontier.pop(0)
            try:
                base_link = urlparse(link).netloc
                
                # check if we are allowed to access the website
                robots_file_loc = "http://" + base_link + "/robots.txt"
                session = requests.Session()
                retry = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])
                adapter = HTTPAdapter(max_retries=retry)
                session.mount('https://', adapter)
                robots_file = session.get(robots_file_loc, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"})
                session.close()
                if robots_file.ok:
                    robot_parser = rp()
                    robot_parser.set_url(robots_file_loc)
                    robot_parser.read()
                    if not robot_parser.can_fetch("*", link):
                        continue
                elif robots_file.status_code != 404:
                    continue
                
                session = requests.Session()
                retry = Retry(total=5, backoff_factor=0.1, status_forcelist=[ 500, 502, 503, 504 ])
                adapter = HTTPAdapter(max_retries=retry)
                session.mount('https://', adapter)
                document = session.get(link, headers={"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}).text
                session.close()
                html5_doc = html5lib.parse(document)
                soup = bs(document, "html.parser")

                # save and use document if it's not a duplicate
                for script in soup(["script", "style"]):
                    script.extract() 

                # check if document is relevant
                doc_lang = html5_doc.get("lang")
                if doc_lang == None:
                    doc_lang = html5_doc.get("xml:lang")
                if doc_lang == None:
                    continue
                relevant = soup.find("body").text.find("Tübingen") > -1 # type: ignore
                if (doc_lang.count("en") > 0 and relevant): # type: ignore
                    # process doc to right format and index
                    doc = {"url": link, "doc": document }
                    # TODO: index document
                    duplicate, docAmount = index(doc, indexPath)
                    if duplicate == False:
                        print("Duplicate")
                        continue
                    # get all links from document and save to frontier if not seen yet
                    for a in soup.find_all('a'):
                        if (a.get('href')):
                            l = a.get('href')
                            if l.startswith('#'):	
                                continue
                            if urlparse(l).netloc == '':
                                l = base_link + l
                            if urlparse(l).scheme == '':
                                l = urlparse(link).scheme + '://' + l
                            if l not in links_seen:
                                frontier.append(l)
                                links_seen.append(l)
                    if docAmount >= MAX_DOCS:
                        break
            except Exception as err:
                if context_manager.state == context_manager.TIMED_OUT:
                    print("Timed out at link: ", link)
                else:
                    print("Exception occured at link: ", link, ". Description: ", err)



In [12]:
#### Execution (crawling, indexing, clustering) ####
## delete content of documents and pictures folder if there is some
# so the process can be started again without resetting everything by hand
docs = [os.path.join(os.getcwd(), "documents", f) for f in os.listdir(os.path.join(os.getcwd(), "documents"))]
for f in docs:
    os.remove(f)

images = [os.path.join(os.getcwd(), "pictures", f) for f in os.listdir(os.path.join(os.getcwd(), "pictures"))]
for i in images:
    os.remove(i)

#write [{}, [], [], None] in index.json 
json_data = json.dumps([{}, [], [], None])
with open(STORAGE_LOC, 'w') as json_file:
    json_file.write(json_data)
    json_file.close()

### crawl ####
    crawl(frontier, STORAGE_LOC)
#after crawling cluster and add remaining infos to index
cluster(STORAGE_LOC)


0
1


KeyboardInterrupt: 

# 2. Query Processing 
Process a textual query and return the 100 most relevant documents from your index. Please incorporate **at least one retrieval model innovation** that goes beyond BM25 or TF-IDF. Please allow for queries to be entered either individually in an interactive user interface (see also #3 below), or via a batch file containing multiple queries at once. The batch file will be formatted to have one query per line, listing the query number, and query text as tab-separated entries. An example of the batch file for the first two queries looks like this:

```
1   tübingen attractions
2   food and drinks
```

In [ ]:
#Retrieve documents relevnt to a query. You need (at least) two parameters:
    #query: The user's search query
    #index: The location of the local index storing the discovered documents.
def retrieve(query, index):
    #TODO: Implement me
    pass

# 3. Search Result Presentation
Once you have a result set, we want to return it to the searcher in two ways: a) in an interactive user interface. For this user interface, please think of **at least one innovation** that goes beyond the traditional 10-blue-links interface that most commercial search engines employ. b) as a text file used for batch performance evaluation. The text file should be formatted to produce one ranked result per line, listing the query number, rank position, document URL and relevance score as tab-separated entries. An example of the first three lines of such a text file looks like this:

```
1   1   https://www.tuebingen.de/en/3521.html   0.725
1   2   https://www.komoot.com/guide/355570/castles-in-tuebingen-district   0.671
1   3   https://www.unimuseum.uni-tuebingen.de/en/museum-at-hohentuebingen-castle   0.529
...
1   100 https://www.tuebingen.de/en/3536.html   0.178
2   1   https://www.tuebingen.de/en/3773.html   0.956
2   2   https://www.tuebingen.de/en/4456.html   0.797
...
```

In [ ]:
#TODO: Implement an interactive user interface for part a of this exercise.

#Produce a text file with 100 results per query in the format specified above.
def batch(results):
    #TODO: Implement me.    
    pass

# 4. Performance Evaluation 
We will evaluate the performance of our search systems on the basis of five queries. Two of them are avilable to you now for engineering purposes:
- `tübingen attractions`
- `food and drinks`

The remaining three queries will be given to you during our final session on July 23rd. Please be prepared to run your systems and produce a single result file for all five queries live in class. That means you should aim for processing times of no more than ~1 minute per query. We will ask you to send carsten.eickhoff@uni-tuebingen.de that file.

# Grading
Your final projects will be graded along the following criteria:
- 25% Code correctness and quality (to be delivered on this sheet)
- 25% Report (4 pages, PDF, explanation and justification of your design choices)
- 25% System performance (based on how well your system performs on the 5 queries relative to the other teams in terms of nDCG)
- 15% Creativity and innovativeness of your approach (in particular with respect to your search system #2 and user interface #3 innovations)
- 10% Presentation quality and clarity

# Permissible libraries
You can use any general-puprose ML and NLP libraries such as scipy, numpy, scikit-learn, spacy, nltk, but please stay away from dedicated web crawling or search engine toolkits such as scrapy, whoosh, lucene, terrier, galago and the likes. Pretrained models are fine to use as part of your system, as long as they have not been built/trained for retrieval. 
